In [ ]:
import pandas as pd

In [ ]:
x_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(x_train_path)

In [ ]:
X_train.head(15)

In [ ]:
X_train.info()


In [ ]:
X_train[["piezo_groundwater_level_category"]]

In [ ]:
X_train.sort_values(["piezo_groundwater_level_category"])


In [ ]:
X_train=X_train.rename({"piezo_groundwater_level_category":"target"},axis=1)
X_train.head()

In [ ]:
X_train.sort_values(["target"])

In [ ]:
X_train[['piezo_station_department_code','target']].sort_values(["target"])



In [ ]:
humid_40=X_train.loc[X_train["target"]=="Low","meteo_humidity_duration_below_40%"]
humid_40.dropna()
humid_40.plot()

In [ ]:
humid_80=X_train.loc[X_train["target"]=="High","meteo_humidity_duration_above_80%"]
humid_40.plot()

In [ ]:
howwet=X_train.loc[X_train["target"]=="Very Low","meteo_wetting_duration"]
howwet.dropna()
